In [2]:
!nvidia-smi

Mon Dec 20 03:03:48 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8    28W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Download custom data



In [3]:
%%shell

gdown --id 1fexVAGt9srIjE4_gHJ043D61AVI-Z5UK
unzip coco_panoptic_subset_trainval.zip > /dev/null
rm -rf coco_panoptic_subset_trainval.zip


Downloading...
From: https://drive.google.com/uc?id=1fexVAGt9srIjE4_gHJ043D61AVI-Z5UK
To: /content/coco_panoptic_subset_trainval.zip
100% 5.37G/5.37G [03:42<00:00, 24.2MB/s]


# Install packages

In [4]:
%%shell

pip install pyyaml==5.1
git clone https://github.com/facebookresearch/detectron2.git
python -m pip install -e detectron2
mv coco detectron2/datasets

git clone https://github.com/dvlab-research/PanopticFCN.git
mv PanopticFCN detectron2/projects/PanopticFCN

     |████████████████████████████████| 274 kB 4.3 MB/s 
  Created wheel for pyyaml: filename=PyYAML-5.1-cp37-cp37m-linux_x86_64.whl size=44092 sha256=a96c1d31f946ae0e3c5c4c5304bc7b747c66503210a434e1055775dc9c569dd0
  Stored in directory: /root/.cache/pip/wheels/77/f5/10/d00a2bd30928b972790053b5de0c703ca87324f3fead0f2fd9
Successfully built pyyaml
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Cloning into 'detectron2'...
remote: Enumerating objects: 13717, done.
remote: Total 13717 (delta 0), reused 0 (delta 0), pack-reused 13717
Receiving objects: 100% (13717/13717), 5.62 MiB | 13.45 MiB/s, done.
Resolving deltas: 100% (9927/9927), done.
Obtaining file:///content/detectron2
     |████████████████████████████████| 49 kB 3.9 MB/s 
     |████████████████████████████████| 74 kB 2.6 MB/s 
     |████████████████████████████████| 145 kB 10.5 MB/s 
     |████████████████████████████████| 130 

In [14]:
!pip install git+https://github.com/cocodataset/panopticapi.git

  Cloning https://github.com/cocodataset/panopticapi.git to /tmp/pip-req-build-90gu6u8d
  Running command git clone -q https://github.com/cocodataset/panopticapi.git /tmp/pip-req-build-90gu6u8d
  Created wheel for panopticapi: filename=panopticapi-0.1-py3-none-any.whl size=8306 sha256=83bb824851817ae1edb0c55d6f51828100775b1018a8f1b34d9098957608ecfc
  Stored in directory: /tmp/pip-ephem-wheel-cache-w0t_4spb/wheels/ad/89/b8/b66cce9246af3d71d65d72c85ab993fd28e7578e1b0ed197f1
Successfully built panopticapi


# Define a training config

In [19]:
%%writefile detectron2/projects/PanopticFCN/configs/PanopticFCN-R50-1x.yaml

MODEL:
  META_ARCHITECTURE: "PanopticFCN"
  WEIGHTS: "detectron2://ImageNetPretrained/MSRA/R-50.pkl"
#   WEIGHTS: "/data/detectron2_modelzoo/basemodel/R-50-MSRA.pkl"
  MASK_ON: True
  PIXEL_MEAN: [123.675, 116.28, 103.53]
  PIXEL_STD: [1.0, 1.0, 1.0]
  RESNETS:
    DEPTH: 50
    OUT_FEATURES: ["res2", "res3", "res4", "res5"]
  FPN:
    IN_FEATURES: ["res2", "res3", "res4", "res5"]
DATASETS:
  TRAIN: ("coco_2017_train_panoptic_separated",)
  TEST: ("coco_2017_val_panoptic_separated",)
DATALOADER:
  FILTER_EMPTY_ANNOTATIONS: True
SOLVER:
  BASE_LR: 0.01
  WEIGHT_DECAY: 1e-4
  LR_SCHEDULER_NAME: "WarmupPolyLR"
  POLY_LR_POWER: 0.9
  WARMUP_ITERS: 1000
  WARMUP_FACTOR: 0.001
  WARMUP_METHOD: "linear"
  CLIP_GRADIENTS:
    ENABLED: True
    CLIP_VALUE: 35.0
  IMS_PER_BATCH: 4
  MAX_ITER: 200
  CHECKPOINT_PERIOD: 10
INPUT:
  MIN_SIZE_TRAIN: (640, 672, 704, 736, 768, 800)
  MIN_SIZE_TRAIN_SAMPLING: "choice"
  MIN_SIZE_TEST: 800
  MAX_SIZE_TRAIN: 1333
  MAX_SIZE_TEST: 1333
  MASK_FORMAT: "bitmask"
VERSION: 2



Overwriting detectron2/projects/PanopticFCN/configs/PanopticFCN-R50-1x.yaml


# Train

In [20]:
%%shell

cd detectron2
python projects/PanopticFCN/train.py --config-file projects/PanopticFCN/configs/PanopticFCN-R50-1x.yaml --num-gpus 1

Command Line Args: Namespace(config_file='projects/PanopticFCN/configs/PanopticFCN-R50-1x.yaml', dist_url='tcp://127.0.0.1:49152', eval_only=False, machine_rank=0, num_gpus=1, num_machines=1, opts=[], resume=False)
[12/20 03:37:05 detectron2]: Rank of current process: 0. World size: 1
[12/20 03:37:06 detectron2]: Environment info:
----------------------  ----------------------------------------------------------------
sys.platform            linux
Python                  3.7.12 (default, Sep 10 2021, 00:21:48) [GCC 7.5.0]
numpy                   1.19.5
detectron2              0.6 @/content/detectron2/detectron2
Compiler                GCC 7.5
CUDA compiler           CUDA 11.1
detectron2 arch flags   3.7
DETECTRON2_ENV_MODULE   <not set>
PyTorch                 1.10.0+cu111 @/usr/local/lib/python3.7/dist-packages/torch
PyTorch debug build     False
GPU available           Yes
GPU 0                   Tesla K80 (arch=3.7)
Driver version          460.32.03
CUDA_HOME               /usr/loca